In [16]:
import pandas as pd
import numpy as np
import os
import math
import json
import tensorflow as tf
import tensorflow_hub as hub


In [17]:
######### Intents #########

intents = {}                                  # Creating a global variable for storing intents
utterance_bag = []                            # Creating global variable to store all the utterances


def create_intent(intent_name):
    if type(intent_name)==list:                # Iterate over all the values in list
        for intent_name in intent_name:
            add_intent(intent_name)

    else:                                      # insert without iterating
        add_intent(intent_name)


def add_intent(intent_name):                   # Function to add intent if not already existing

    if not check_intent_name(intent_name):
        intents[intent_name] = []
    else:
        print("Intent {0} already exists".format(intent_name))

def check_intent_name(intent_name):            # Checking if an intent already exists
    if intent_name in intents.keys():
        return True

    else:
        return False

def clear():                                   # Clearing all the variables
    intents.clear()
    utterance_bag.clear()



In [18]:
######## Utterances #########

## Pass an elememt or list of utterances to be a added for a given intent ##
## If intent does not exists, then intent will be created and then utteracnes will be added ##

def create_utterance(intent_name, utterance):

    if intent_name in intents.keys():                   # Check if intent already exists, add utterance only if it exists
        if type(utterance) == list:
            for utterance in utterance:
                add_utterance(intent_name, utterance)

        else:
            add_utterance(intent_name, utterance)

    else:
        create_intent(intent_name)                     # If utterance doesnt exists, then call create intent and then call itself to add utterance
        create_utterance(intent_name, utterance)


def add_utterance(intent_name, utterance):
    if not check_utterance(utterance):
        intents[intent_name].append(utterance)
        utterance_bag.append(utterance)
    else:
        print("Utterance {0} already exists".format(utterance))

def check_utterance(utterance):
    if utterance in utterance_bag:                    # Checking the utterance in the bag of utterances. If it exists in any intent, it will give an error
        return True
    else:
        return False


In [19]:
########## Delete intents/utterances ##########

## pass intent name to be deleted ##
def remove_intent(intent_name):                       # Removes specified intent
    if type(intent_name) == list:
        for intent_name in intent_name:
            try:
                intents.pop(intent_name)
            except KeyError:
                print("'{0}' intent doesnt exists".format(intent_name))      # Throws an exception if intent does not exists

    else:
        try:
            intents.pop(intent_name)
        except KeyError:
            print("'{0}' intent doesnt exists".format(intent_name))


## Pass an utterance or list of utterances to be deleted along with intent ##
# Todo - identify intent name without user specifying

def remove_utterance(intent_name, utterance):           # removes utterances
    if type(utterance) == list:
        for utterance in utterance:
            try:
                intents[intent_name].remove(utterance)
            except ValueError:
                print("'{0}' utterance doesnt exists".format(utterance)) # throws exception if utterance does not exists

    else:
        try:
            intents[intent_name].remove(utterance)
        except ValueError:
            print("'{0}' utterance doesnt exists".format(utterance))



In [20]:
######### Storing/Retrieving data ############
def store_data():
    with open("sample.json", "w") as file:
        json.dump(intents, file)

def retrieve_data():
    with open("sample.json", "r") as file:
        intents.update(json.load(file))




In [21]:
######### Get intent from utterance ########
def get_intent(utterance):
    for k, v in intents.items():
        if utterance in v:
            return k


In [22]:
def match_intent(sent):
    utterance_list = []
    for k,v in intents.items():
        utterance_list = utterance_list + v['utterance']
    module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
    embed = hub.Module(module_url)
    sentences = [sent]+utterance_list
    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_sentences_encodings = embed(similarity_input_placeholder)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        sentences_embeddings = session.run(similarity_sentences_encodings, feed_dict={similarity_input_placeholder: sentences})
        input_embed = sentences_embeddings[0]
        utterance_embed = sentences_embeddings[1:]
        max = -2
        for index, s in enumerate(utterance_embed):
            sim = np.inner(input_embed,s)
            if(sim > max):
                max = sim
                max_index = index
            matched_utterance = utterance_list[max_index+1]
        print(get_intent(matched_utterance))#USE THIS UTTERANCE TO GET THE INTENT AS THIS IS THE UTTERANCE WITH MAXIMUM SIMILARITY   



In [23]:
############## Testing ##############
def test():
    create_utterance("greeting", ["hi", "hey", "yo"])
    create_intent(["introduction", "aboutyourself"])
    create_utterance("introduction", "i am karthik")
    create_utterance("aboutyourself", ["what do you do?", "where did you work?", "i worked as an engineer", "what do you do for living"])
    create_utterance("story", ["i am interesetd in hearing your story","tell me your story", "what happened"])
    create_utterance("family", ["you got a family", "were you married", "do you have kids?", "who else lives with you?"])
    create_utterance("holiday", ["do you go out for holidays?", "which was your last vacation", "do you like to visit different places?"])

    # Send as json file
    store_data()

    # Printing the values
    print("intents after testing with add test functions{0}".format(intents))

    # Testing remove functions
    remove_intent("story")
    remove_utterance("family", "tell me your story")
    remove_utterance("family", "Remove an utterance which does not exists")
    remove_utterance("greeting", "hi")

    print("intents after deleting test functions{0}".format(intents))

    retrieve_data()

    print("intents after retriving data from json file{0}".format(intents))



In [25]:
clear()

In [26]:
intents


{}

In [29]:
test()

Utterance hi already exists
Utterance hey already exists
Utterance yo already exists
Intent introduction already exists
Intent aboutyourself already exists
Utterance i am karthik already exists
Utterance what do you do? already exists
Utterance where did you work? already exists
Utterance i worked as an engineer already exists
Utterance what do you do for living already exists
Utterance i am interesetd in hearing your story already exists
Utterance tell me your story already exists
Utterance what happened already exists
Utterance you got a family already exists
Utterance were you married already exists
Utterance do you have kids? already exists
Utterance who else lives with you? already exists
Utterance do you go out for holidays? already exists
Utterance which was your last vacation already exists
Utterance do you like to visit different places? already exists
intents after testing with add test functions{'greeting': ['hi', 'hey', 'yo'], 'introduction': ['i am karthik'], 'aboutyoursel

In [30]:
intents

{'greeting': ['hi', 'hey', 'yo'],
 'introduction': ['i am karthik'],
 'aboutyourself': ['what do you do?',
  'where did you work?',
  'i worked as an engineer',
  'what do you do for living'],
 'family': ['you got a family',
  'were you married',
  'do you have kids?',
  'who else lives with you?'],
 'holiday': ['do you go out for holidays?',
  'which was your last vacation',
  'do you like to visit different places?'],
 'story': ['i am interesetd in hearing your story',
  'tell me your story',
  'what happened']}

In [31]:
match_intent("do you have vacations?")

TypeError: list indices must be integers or slices, not str